In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats
import seaborn as sns
from pydataset import data
from fredstats import get_db_url, make_df_from_db
from env import host, user, password

In [2]:
#Make the normal curve for the given data
office1 = stats.norm(90, 15)
office2 = stats.norm(100, 20)

In [3]:
#We're comparing two groups, so we'll nead there means and std-devs.
xbar1 = office1.mean()
xbar2 = office2.mean()

#Trial size was given in question
n1 = 40
n2 = 50

s1 = office1.std()
s2 = office2.std()

In [4]:
#Taking a little off
degf = n1 + n2 - 2

In [5]:
#This one is a doozy.
s_p = sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))

In [6]:
#This might be unecessary for this question.
standard_error = se = sqrt(s1**2 / n1 + s2**2 / n2)

In [7]:
#This is the score we're going to find out.
#This is that crazy long 2-Sample T Test formula written out in Python
t = (xbar1 - xbar2) / (s_p * sqrt(1/n1 + 1/n2))


In [14]:
t

-2.6252287036468456

In [17]:
#more than .99 of t-values are higher than this t score
stats.t(degf).sf(t)

0.9948950737753803

In [18]:
stats.t(degf).cdf(t)

0.005104926224619695

In [19]:
stats.t(degf).pdf(t)

0.013882685754167083

In [ ]:
#Got these from an online 2-sample T-Test app
# Equal Variances
# Pooled Standard Deviation: 17.9567
# Pooled DF: 88
# 95% Confidence Interval for the Difference ( -17.3355 , -2.6645 )

# Test Statistic t= -2.6252
# Population 1 ≠ Population 2: P-Value = 0.0102
# Population 1 < Population 2: P-Value = 0.9949
# Population 1 > Population 2: P-Value = 0.0051

In [22]:
mpg = data('mpg')

In [23]:
mpg.trans = mpg.trans.str[:-4]

In [26]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto,f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual,f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual,f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto,f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto,f,16,26,p,compact


In [35]:
pop_city_mileage = mpg.cty.mean()
auto_city_mileage = mpg[mpg.trans == 'auto'].cty

In [36]:
stats.ttest_1samp(auto_city_mileage, pop_city_mileage)


Ttest_1sampResult(statistic=-2.9001322447463838, pvalue=0.004268839797630685)

In [37]:
#Do volkswagens get better mileage than the all other cars on the highway?
#null = Volksagen.mean == all_cars.mean

In [39]:
volks_hwy_mileage = mpg[mpg.manufacturer == 'volkswagen'].hwy

In [42]:
mpg.hwy.mean()


23.44017094017094

In [43]:
stats.ttest_1samp(volks_hwy_mileage, mpg.hwy.mean())

Ttest_1sampResult(statistic=5.652041311757083, pvalue=6.068818710469792e-06)

Is there a linear relationship between city and highway mileage?



In [45]:
x = mpg.hwy

In [46]:
y = mpg.cty

In [47]:
r, p = stats.pearsonr(x,y)

In [48]:
r

0.9559159136495723

In [49]:
p

1.8683071952618993e-125

Is there a linear relationship between engine displacment and city mileage?

In [50]:
X = mpg.displ
Y = mpg.cty

In [51]:
R, P = stats.pearsonr(X,Y)

In [52]:
R

-0.7985239689348551

In [53]:
P

4.737914890205637e-53

There is a negative correlation that is statistically significant between engine displacement and city mileage.

CORRELATIONS

1.
Use the telco_churn data.  

In [44]:
url = get_db_url(user, host, password, "telco_churn")
df = make_df_from_db(url, 'customers')

Does tenure correlate with monthly charges?   service?

In [88]:
#correlation between tenure and monthly charges
corr, pvalue = stats.pearsonr(df.tenure, df.monthly_charges)



In [87]:
print(f"The correlation is at {corr} and the p-value is {pvalue}, so we can reject the null hypothesis.")

The correlation is at 0.24789985628615013 and the p-value is 4.0940449915016345e-99, so we can reject the null hypothesis.


Total charges?

In [30]:
df.total_charges.sort_values(ascending=True).head(11)

2855     
3118     
3052     
5433     
1731     
4054     
2176     
2250     
945      
1906     
2025     
Name: total_charges, dtype: object

In [33]:
df.iloc[2855].total_charges

' '

In [47]:
df = df[df.total_charges != ' ']

In [50]:
df.total_charges = df.total_charges.astype(float)

In [53]:
corr, p = stats.pearsonr(df.total_charges, df.monthly_charges)

In [57]:
print(f"The correlation is at {corr} and the p-value is {p}, so we can reject the null hypothesis.")

The correlation is at 0.6510648032262025 and the p-value is 0.0, so we can reject the null hypothesis.


What happens if you control for phone and internet

2. Use the employees database.



In [6]:
url = get_db_url(user, host, password, "employees")
#Gonna use Kevin's custom SQL select to get all the data in one DF

In [7]:
employees = pd.read_sql('''SELECT
    e.*,
    datediff('2002-09-30', e.hire_date) tenure,
    t.title,
    t.from_date title_from,
    datediff('2002-09-30', t.from_date) t_tenure,
    et.titles,
    s.salary,
    s.from_date salary_from,
    datediff('2002-09-30', s.from_date) s_tenure,
    es.salaries
FROM
    employees e
JOIN 
    titles t 
    USING(emp_no)
JOIN 
    salaries s 
    USING(emp_no)
JOIN
    (SELECT 
        emp_no,
        count(*) titles
    FROM
        titles
    GROUP BY
        emp_no) et
    USING(emp_no)
JOIN
    (SELECT 
        emp_no,
        count(*) salaries
    FROM
        salaries
    GROUP BY
        emp_no) es
    USING(emp_no)
WHERE
    s.to_date > '2002-09-30'
    AND t.to_date > '2002-09-30';
''', url)

In [13]:
employees.head()
#Kevin already has a tenure column. So lets just find correlation between that and salary!

,emp_no,birth_date,first_name,last_name,gender,hire_date,tenure,title,title_from,t_tenure,titles,salary,salary_from,s_tenure,salaries
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,5940,Senior Engineer,1986-06-26,5940,1,88958,2002-06-22,100,17
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,6157,Staff,1996-08-03,2249,1,72527,2001-08-02,424,6
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,5877,Senior Engineer,1995-12-03,2493,1,43311,2001-12-01,303,7
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,5782,Senior Engineer,1995-12-01,2495,2,74057,2001-11-27,307,16
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,4766,Senior Staff,1996-09-12,2209,2,94692,2001-09-09,386,13


In [135]:
corr, pvalue = stats.pearsonr(employees.tenure, employees.salary)


Is there a relationship between how long an employee has been with the company and their salary?

In [138]:
print(f"The correlation is at {corr}, which is less than what I'd call a correlation, and the p-value is {pvalue}.")

The correlation is at 0.3064625613186075, which is less than what I'd call a correlation, and the p-value is 0.0.


Is there a relationship between how long an employee has been with the company and the number of titles they have had?

In [139]:
corr, pvalue = stats.pearsonr(employees.tenure, employees.titles)

In [140]:
print(f"The correlation is at {corr}, which is less than what I'd call a correlation, and the p-value is {pvalue}.")

The correlation is at 0.28712678896997174, which is less than what I'd call a correlation, and the p-value is 0.0.


3.Use the sleepstudy data. Is there a relationship between days and reaction time?

In [14]:
sleep_data = data('sleepstudy')

In [22]:
corr, pvalue = stats.pearsonr(sleep_data.Days, sleep_data.Reaction)

In [23]:
print(f"The correlation is middling at {corr}, but {pvalue} is too small not to reject H0.")

The correlation is middling at 0.5352302262650254, but 9.89409632221453e-15 is too small not to reject H0.


CHI-SQUARE TEST

1. Use the following contingency table to help answer the question of whether using a macbook and being a codeup student are independent of each other.

In [141]:
index=['Has Macbook','No Macbook']
columns=['Codeup Student','Not Student']

In [142]:
observations = pd.DataFrame([[49,20],[1,30]],index=index, columns=columns)

In [145]:
chi2, p, degf, expected = stats.chi2_contingency(observations)

In [146]:
p

1.4116760526193828e-09

In [147]:
print(f"The {p} is very low, so it'd be safe to reject the null hypothesis that these observed values are evenly distributed.")

The 1.4116760526193828e-09 is very low, so it'd be safe to reject the null hypothesis that these observed values are evenly distributed.


2.Choose another 2 categorical variables from the mpg dataset and perform a 
c
h
i
2
 contingency table test with them. Be sure to state your null and alternative hypotheses.



In [171]:
#Crosstab to find the observed values of two categorical variables.
#This can be plugged into the stats.chi2_contigency() function to get the
#chi score, pvalue, degreeOfFreedom and the expected values
car_observations = pd.crosstab(mpg.year, mpg.trans)

Null Hypothesis: There is no correlation between the they year a car was made and their transmission.
Alt Hypothesis:  There is a correlation between the two. THEY ARE NOT SIGNIFICANT!

In [172]:
#117 cars in two years, so there ouugth to be a 50/50 split if it were evenly distributed
car_observations

trans,auto,manual
year,,
1999,74,43
2008,83,34


In [165]:
chi2, p, degf, expected = stats.chi2_contingency(car_observations)

In [166]:
expected

array([[78.5, 38.5],
       [78.5, 38.5]])

In [173]:
p

0.2657001056392438

In [174]:
print(f"The p-value of {p} is large enough for the null hypothesis to not be rejected")

The p-value of 0.2657001056392438 is large enough for the null hypothesis to not be rejected


3.Use the data from the employees database to answer these questions:




In [2]:
url = get_db_url(host,user,password,'employees')
#For some reason, I can't run these SQL queries first. I have to use the one from the previous
#look through the employees DB.

In [8]:
emp_df = pd.read_sql_table('employees', url)

In [9]:
de_df = pd.read_sql_table('dept_emp', url)

In [10]:
d_df = pd.read_sql_table('departments', url)

In [9]:
#join them

In [11]:
df_merged = pd.merge(emp_df, de_df, how='inner', on = 'emp_no')

In [12]:
df_merged = pd.merge(df_merged, d_df, how='inner', on = 'dept_no')

In [ ]:
#they're merged

In [13]:
df_merged.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,dept_no,from_date,to_date,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,d005,1986-06-26,NaT,Development
1,10006,1953-04-20,Anneke,Preusig,F,1989-06-02,d005,1990-08-05,NaT,Development
2,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15,d005,1998-03-11,2000-07-31,Development
3,10012,1960-10-04,Patricio,Bridgland,M,1992-12-18,d005,1992-12-18,NaT,Development
4,10014,1956-02-12,Berni,Genin,M,1987-03-11,d005,1993-12-29,NaT,Development


In [20]:
# making boolean series for a team sales 
my_way = df_merged[(df_merged['dept_name'] == "Sales") | (df_merged['dept_name'] == "Marketing")]


In [40]:
#Zach did it this way.
sales_and_marketing = df_merged[df_merged.dept_name.isin(['Sales','Marketing'])]

In [43]:
#Only current employees. So their to-date should be a null value

sales_and_marketing = sales_and_marketing[sales_and_marketing.to_date.isnull()]

In [17]:
observos = pd.crosstab(sales_and_marketing.gender, sales_and_marketing.dept_name)

In [18]:
chi2, p, degfree, expected = stats.chi2_contingency(observos)

Is an employee's gender independent of whether an employee works in sales or marketing? (only look at current employees)

In [19]:
print(f"The x2 score is {chi2} and the p-value is {p}.")

The x2 score is 0.3240332004060638 and the p-value is 0.5691938610810126.


Is an employee's gender independent of whether or not they are or have been a manager?

In [45]:
dm_df = pd.read_sql_table('dept_manager', url)

In [74]:
manag_exp = np.ones(24)

In [75]:
dm_df['man_exp'] = manag_exp

In [76]:
dm_df.head()

,emp_no,dept_no,from_date,to_date,man_exp
0,110022,d001,1985-01-01,1991-10-01,1.0
1,110039,d001,1991-10-01,NaT,1.0
2,110085,d002,1985-01-01,1989-12-17,1.0
3,110114,d002,1989-12-17,NaT,1.0
4,110183,d003,1985-01-01,1992-03-21,1.0


In [77]:
df_managers = pd.merge(emp_df, dm_df, how='left', on = 'emp_no')

In [86]:
#I had to fill in the null values with zeros so they could be evaluated for hypothesis testing.
df_managers.man_exp = df_managers.man_exp.fillna(0)

In [88]:
df_managers.man_exp.sum()

24.0

In [90]:
manager_observos = pd.crosstab(df_managers.gender, df_managers.man_exp)

In [91]:
manager_observos

man_exp,0.0,1.0
gender,,
F,120038,13
M,179962,11


In [92]:
chi2, p, degfree, expected = stats.chi2_contingency(manager_observos)

In [93]:
expected

array([[1.20041397e+05, 9.60331174e+00],
       [1.79958603e+05, 1.43966883e+01]])

In [94]:
print(f"The x2 score is {chi2} and the p-value is {p}.")

The x2 score is 1.4566857643547197 and the p-value is 0.22745818732810363.


In [95]:
560 + 650

1210